In [ ]:
import subprocess
import json
import pandas as pd
import os

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

# statsperform API + bq backloading

In [ ]:
import importlib
import data
importlib.reload(data)
from data import DataLoader

prem2425 = '9n12waklv005j8r32sfjj2eqc'

prem_loader = DataLoader(prem2425)

# feature engineering

## data loading

In [ ]:
def get_all_team_data():
        query = f"""
                select *
                from prizepicksanalytics.soccer_simulations.team_match_data
                """
        
        return prem_loader.execute_bq_query(query)

def get_all_player_data():
        query = f"""
                select *
                from prizepicksanalytics.soccer_simulations.player_match_data
                """
        
        return prem_loader.execute_bq_query(query)

def get_historic_team_data(match):

        home_query = f"""
                select *
                from prizepicksanalytics.soccer_simulations.team_match_data
                where team_id = '{match.home_id}'
                and date(REPLACE(match_date, 'Z', '')) < date('{match.localDate}')
                """
        
        away_query = f"""
                select *
                from prizepicksanalytics.soccer_simulations.team_match_data
                where team_id = '{match.away_id}'
                and date(REPLACE(match_date, 'Z', '')) < date('{match.localDate}')
                """


        return prem_loader.execute_bq_query(home_query),   prem_loader.execute_bq_query(away_query)


# def get_historic_player_data(match,
#                              list_of_player_ids):
        
#         player_query = f"""
#                 select *
#                 from prizepicksanalytics.soccer_simulations.player_match_data
#                 where playerId in ({", ".join(["'" + x + "'" for x in list_of_player_ids])})
#                 and date(REPLACE(match_date, 'Z', '')) < date('{match.localDate}')
#                 """


#         return prem_loader.execute_bq_query(player_query)

        # return player_query

def get_squads(match):

        home_query = f"""
                select *
                from prizepicksanalytics.soccer_simulations.squad_data
                where contestantId = '{match.home_id}'
                """
        
        away_query = f"""
                select *
                from prizepicksanalytics.soccer_simulations.squad_data
                where contestantId = '{match.away_id}'
                """


        return prem_loader.execute_bq_query(home_query), prem_loader.execute_bq_query(away_query)


def get_calendar():
    query = """
        select * from prizepicksanalytics.soccer_simulations.match_calendar
    """
    return prem_loader.execute_bq_query(query)

In [ ]:


# away_query = f"""
#                 select *
#                 from prizepicksanalytics.soccer_simulations.squad_data
#                 """


# squad_data = prem_loader.execute_bq_query(away_query)

# calendar_data = get_calendar()
# calendar_data.to_csv('calendar_data.csv', index=False)


# tourneys = get_statsperform_tourneys()
# team_data = get_all_team_data()
# team_data.to_csv('team_data.csv', index=False)
# player_data = get_all_player_data()
# player_data.to_csv('player_data.csv', index=False)
# ars_lfc_match = prem_loader.matches.iloc[290]

## agg methods

In [ ]:
def _get_team_stats_vector(team_data,
                           opposite_team_data,
                           position):
    
    home = True if position == 'home' else False
    position_performances = team_data[team_data['home'] == home]

    if team_data.shape[0] == 0:
        avg_possession = np.nan
        avg_finalThirdEntries = np.nan
        avg_accurateBackZonePass = np.nan
        avg_accurateFwdZonePass = np.nan

        avg_possWonAtt3rd = np.nan
        avg_possWonMid3rd = np.nan
        avg_interception = np.nan
    else:
        avg_possession = team_data.possessionPercentage.mean()
        avg_finalThirdEntries = team_data.finalThirdEntries.mean()
        avg_accurateBackZonePass = team_data.accurateBackZonePass.mean()
        avg_accurateFwdZonePass = team_data.accurateFwdZonePass.mean()

        avg_possWonAtt3rd = team_data.possWonAtt3rd.mean()
        avg_possWonMid3rd = team_data.possWonMid3rd.mean()
        avg_interception = team_data.interception.mean()

    if position_performances.shape[0] == 0:
        ps_possession = np.nan
        ps_finalThirdEntries = np.nan
        ps_accurateBackZonePass = np.nan
        ps_accurateFwdZonePass = np.nan

        ps_possWonAtt3rd = np.nan
        ps_possWonMid3rd = np.nan
        ps_interception = np.nan
    else:
        ps_possession = position_performances.possessionPercentage.mean()
        ps_finalThirdEntries = position_performances.finalThirdEntries.mean()
        ps_accurateBackZonePass = position_performances.accurateBackZonePass.mean()
        ps_accurateFwdZonePass = position_performances.accurateFwdZonePass.mean()

        ps_possWonAtt3rd = position_performances.possWonAtt3rd.mean()
        ps_possWonMid3rd = position_performances.possWonMid3rd.mean()
        ps_interception = position_performances.interception.mean()
    
    
    return {
        'avg_possession': avg_possession,
        'avg_finalThirdEntries': avg_finalThirdEntries,
        'avg_accurateBackZonePass': avg_accurateBackZonePass,
        'avg_accurateFwdZonePass': avg_accurateFwdZonePass,
        'avg_possWonAtt3rd': avg_possWonAtt3rd,
        'avg_possWonMid3rd': avg_possWonMid3rd,
        'avg_interception': avg_interception,

        'ps_possession': ps_possession,
        'ps_finalThirdEntries': ps_finalThirdEntries,
        'ps_accurateBackZonePass': ps_accurateBackZonePass,
        'ps_accurateFwdZonePass': ps_accurateFwdZonePass,
        'ps_possWonAtt3rd': ps_possWonAtt3rd,
        'ps_possWonMid3rd': ps_possWonMid3rd,
        'ps_interception': ps_interception,
    }

def _get_player_stats(player_performances,
                      position):
    home = True if position == 'home' else False
    position_performances = player_performances[player_performances['home'] == home]

    if team_data.shape[0] == 0:
        avg_passAtt = np.nan
        avg_minsPlayed = np.nan
        # avg_finalThirdEntries = np.nan
        # avg_accurateBackZonePass = np.nan
        # avg_accurateFwdZonePass = np.nan

        # avg_possWonAtt3rd = np.nan
        # avg_possWonMid3rd = np.nan
        # avg_interception = np.nan
    else:
        avg_passAtt = player_performances.totalPass.mean()
        avg_minsPlayed = player_performances.minsPlayed.mean()
        # avg_finalThirdEntries = team_data.finalThirdEntries.mean()
        # avg_accurateBackZonePass = team_data.accurateBackZonePass.mean()
        # avg_accurateFwdZonePass = team_data.accurateFwdZonePass.mean()

        # avg_possWonAtt3rd = team_data.possWonAtt3rd.mean()
        # avg_possWonMid3rd = team_data.possWonMid3rd.mean()
        # avg_interception = team_data.interception.mean()

    if position_performances.shape[0] == 0:
        ps_passAtt = np.nan
        ps_minsPlayed = np.nan
        # ps_finalThirdEntries = np.nan
        # ps_accurateBackZonePass = np.nan
        # ps_accurateFwdZonePass = np.nan

        # ps_possWonAtt3rd = np.nan
        # ps_possWonMid3rd = np.nan
        # ps_interception = np.nan
    else:
        ps_passAtt = position_performances.totalPass.mean()
        ps_minsPlayed = position_performances.minsPlayed.mean()
        # ps_finalThirdEntries = position_performances.finalThirdEntries.mean()
        # ps_accurateBackZonePass = position_performances.accurateBackZonePass.mean()
        # ps_accurateFwdZonePass = position_performances.accurateFwdZonePass.mean()

        # ps_possWonAtt3rd = position_performances.possWonAtt3rd.mean()
        # ps_possWonMid3rd = position_performances.possWonMid3rd.mean()
        # ps_interception = position_performances.interception.mean()
    
    
    return {
        # 'player_id':player_performances.matchName.iloc[0],
        'avg_passes': avg_passAtt,
        'avg_minsPlayed': avg_minsPlayed,
        # 'avg_accurateBackZonePass': avg_accurateBackZonePass,
        # 'avg_accurateFwdZonePass': avg_accurateFwdZonePass,
        # 'avg_possWonAtt3rd': avg_possWonAtt3rd,
        # 'avg_possWonMid3rd': avg_possWonMid3rd,
        # 'avg_interception': avg_interception,

        'ps_passes': ps_passAtt,
        'ps_minsPlayed': ps_minsPlayed,
        # 'ps_accurateBackZonePass': ps_accurateBackZonePass,
        # 'ps_accurateFwdZonePass': ps_accurateFwdZonePass,
        # 'ps_possWonAtt3rd': ps_possWonAtt3rd,
        # 'ps_possWonMid3rd': ps_possWonMid3rd,
        # 'ps_interception': ps_interception,
    }

## get match vectors

In [ ]:
import numpy as np

    
def get_match_vectors(match,
                      all_performances = None,
                      all_squads=None,
                      all_player_data=None,
                      training=True):

    final_feat_list = []

    team_ids = {'home':match.home_id,
                'away':match.away_id}

    if all_performances is None:
        home_data, away_data = get_historic_team_data(match)
    else:
        home_data = all_performances[(all_performances['team_id'] == team_ids['home']) &
                                     (all_performances['match_date'].apply(lambda x: x.replace('Z', '')) < match.localDate)]
        
        away_data = all_performances[(all_performances['team_id'] == team_ids['away']) &
                                     (all_performances['match_date'].apply(lambda x: x.replace('Z', '')) < match.localDate)]

    team_data = {
        'home':home_data,
        'away':away_data
    }

    if all_squads is None:
        home_squad, away_squad = get_squads(match)
    else:
        home_squad, away_squad = all_squads[team_ids['home']], all_squads[team_ids['away']]

    squad_data = {
        'home':home_squad,
        'away':away_squad
    }

    for team in ['home', 'away']:
        opposite_position = 'away' if team == 'home' else 'home'

        team_stats_vector = _get_team_stats_vector(team_data[team],
                                                      team_data[opposite_position],
                                                      team)
        
        # team_name = team_data[team].iloc[0].team_name
        
        team_squad = squad_data[team]

        if all_player_data is None:
            historical_player_data = get_historic_player_data(match, list(team_squad.id.unique()))
        else: 
            historical_player_data = all_player_data[(all_player_data['playerId'].isin(team_squad.id.unique())) &
                                                      (all_player_data['match_date'].apply(lambda x: x.replace('Z', '')) < match.localDate)]
            
        team_squad = team_squad[team_squad['id'].isin(historical_player_data.playerId.unique())]
        # team_squad = all_player_data[(all_player_data['team_id'] == team_ids[team]) & (all_player_data['match_id'] == match.id)]

        for i, player in team_squad.iterrows():

            final_dict = {
                'player_id':player.id,
                'player_name':player.matchName,
                'player_position':player.position,
                'team_id':team_ids[team],
                'match_id':match.id,
                'match_date':match.localDate,
                'match_week':match.week
            }

            # print('player id', player.playerId)

            player_performances = historical_player_data[historical_player_data['playerId'] == player.id]
            player_vector = _get_player_stats(player_performances,
                                              team)
            
            final_dict.update(player_vector)
            final_dict.update(team_stats_vector)          

            if training == True:
                try:
                    mask = (all_player_data['match_id'] == match.id) & (all_player_data['playerId'] == player.id)
                    final_dict['target'] = all_player_data[mask].iloc[0].totalPass
                except IndexError as e:
                    continue
        
            # player_vector['match_id'] = match.id
            # player_vector['match_date'] = match.localDate
            # player_vector['match_week'] = match.week

            final_feat_list.append(final_dict)
        
    return final_feat_list

## feature generation

In [ ]:
import pickle
calendar = pd.read_csv('match_calendar.csv')

team_data = pd.read_csv('team_data.csv')
team_data = team_data.sort_values(['match_date'], ascending=True).reset_index(drop=True)

player_data = pd.read_csv('player_data.csv')
player_data = player_data.sort_values(['match_date', 'match_id', 'team_id', 'playerId'], ascending=True).reset_index(drop=True)

with open('squads.pickle', 'rb') as fila:
    all_squads = pickle.load(fila)



list_of_all_vectors = []

for i, match in calendar.iterrows():

    vectors = get_match_vectors(match,
                    all_performances=team_data,
                    all_squads=all_squads,
                    all_player_data=player_data,
                    training=True)
                    

    list_of_all_vectors.extend(vectors)

feature_list = pd.DataFrame(list_of_all_vectors)
feature_list.to_csv('./v1features.csv')

# modeling

In [ ]:
features = pd.read_csv('v1features.csv', index_col=0)
features = features.dropna()
features = features.sort_values('match_date', ascending=True)

weeks = features.match_week.unique()
test_split = int(len(weeks) * 0.8)

train_weeks = weeks[:test_split]
test_weeks = weeks[test_split:]

train = features[features['match_week'].isin(train_weeks)]
test = features[features['match_week'].isin(test_weeks)]


config_cols = ['player_name', 'player_position', 'team_id', 'match_id', 'match_date', 'match_week', 'player_id']
feats = train.drop(columns=config_cols + ['target']).columns
target = ['target']

In [ ]:
train.head()

## alexboost

In [ ]:
from alexboost import Normal

In [ ]:
norm = Normal(
        num_trees=50, 
        num_samples_per_bucket=30,
    )


norm.fit(train[feats], train[target].target)

In [ ]:
preds = norm.predict(test[feats])
test[['exp_mean', 'exp_std']] = preds
test['mae_error'] = (test.target - preds[:, 0])

# evaluation

## misc

In [ ]:
test.groupby('player_position')['mae_error'].sum().sort_values()

In [ ]:
test.groupby('player_position')['mae_error'].apply(lambda x: (sum(x)) / len(x)).sort_values()

In [ ]:
test.groupby('team_id')['error'].apply(lambda x: (sum(x)) / len(x)).sort_values()

In [ ]:
import seaborn as sns # Often imported as sns

# test.exp_std.plot()
sns.scatterplot(
    data=test[test['error'] < 60].iloc[:1000],
    x='error',
    y='avg_minsPlayed',
    hue='player_position'
)

In [ ]:
norm.regressor.feature_importances_
pd.DataFrame({"feature": train[feats].columns, "importance": norm.regressor.feature_importances_}).sort_values(by="importance", ascending=False).round(3)

In [ ]:
import matplotlib.pyplot as plt
train[target].target.hist()

In [ ]:
train.sort_values('target', ascending=True)